# Table of Contents
 <p><div class="lev1"><a href="#Clustering-and-k-means"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering and k-means</a></div><div class="lev1"><a href="#Generating-Samples"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generating Samples</a></div><div class="lev1"><a href="#Initialisation"><span class="toc-item-num">3&nbsp;&nbsp;</span>Initialisation</a></div>

# Clustering and k-means

We now venture into our first application, which is clustering with the k-means algorithm. Clustering is a data mining exercise where we take a bunch of data and find groups of points that are similar to each other. K-means is an algorithm that is great for finding clusters in many types of datasets.

# Generating Samples

First up, we are going to need to generate some samples. We could generate the samples randomly, but that is likely to either give us very sparse points, or just one big group - not very exciting for clustering.

Instead, we are going to start by generating three centroids, and then randomly choose (with a normal distribution) around that point. First up, here is a method for doing this:



In [27]:
%%writefile functions.py

import tensorflow as tf
import numpy as np


def create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed):
    np.random.seed(seed)
    slices = []
    centroids = []
    # Create samples for each cluster
    for i in range(n_clusters):
        samples = tf.random_normal((n_samples_per_cluster, n_features),
                               mean=0.0, stddev=5.0, dtype=tf.float32, seed=seed, name="cluster_{}".format(i))
        current_centroid = (np.random.random((1, n_features)) * embiggen_factor) - (embiggen_factor/2)
        centroids.append(current_centroid)
        samples += current_centroid
        slices.append(samples)
    # Create a big "samples" dataset
    samples = tf.concat(0, slices, name='samples')
    centroids = tf.concat(0, centroids, name='centroids')
    return centroids, samples


Overwriting functions.py


The way this works is to create **n_clusters different** centroids at random (using **np.random.random((1, n_features))**) and using those as the centre points for **tf.random_normal**. The **tf.random_normal** function generates normally distributed random values, which we then add to the current centre point. This creates a blob of points around that center. We then record the centroids (**centroids.append**) and the generated samples (**slices.append(samples)**). Finally, we create “One big list of samples” using **tf.concat**, and convert the centroids to a TensorFlow Variable as well, also using **tf.concat**.

Saving this **create_samples** method in a file called **functions.py** allows us to import these methods into our scripts for this (and the next!) lesson. Create a new file called **generate_samples.py**, which has the following code:

In [28]:
%%writefile generate_samples.py

import tensorflow as tf
import numpy as np

from functions import create_samples

n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70

np.random.seed(seed)

centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)

model = tf.initialize_all_variables()
with tf.Session() as session:
    sample_values = session.run(samples)
    centroid_values = session.run(centroids)

Overwriting generate_samples.py


This just sets up the number of clusters and features (I recommend keeping the number of features at 2, allowing us to visualise them later), and the number of samples to generate. Increasing the [embiggen_factor](https://en.wiktionary.org/wiki/embiggen) will increase the “spread” or the size of the clusters. I chose a value here that provides good learning opportunity, as it generates visually identifiable clusters.

To visualise the results, lets create a plotting function using matplotlib. Add this code to functions.py:

In [29]:
%%writefile -a functions.py


def plot_clusters(all_samples, centroids, n_samples_per_cluster):
    import matplotlib.pyplot as plt
    # Plot out the different clusters
    # Choose a different colour for each cluster
    colour = plt.cm.rainbow(np.linspace(0,1,len(centroids)))
    for i, centroid in enumerate(centroids):
        # Grab just the samples fpr the given cluster and plot them out with a new colour
        samples = all_samples[i*n_samples_per_cluster:(i+1)*n_samples_per_cluster]
        plt.scatter(samples[:,0], samples[:,1], c=colour[i])
        # Also plot centroid
        plt.plot(centroid[0], centroid[1], markersize=35, marker="x", color='k', mew=10)
        plt.plot(centroid[0], centroid[1], markersize=30, marker="x", color='m', mew=5)
    plt.show()


Appending to functions.py


All this code does is plots out the samples from each cluster using a different colour, and creates a big magenta X where the centroid is. The centroid is given as an argument, which will be handy later on.

Update the **generate_samples.py** to import this function by adding **from functions import plot_clusters** to the top of the file. Then, add this line of code to the bottom:

```python
plot_clusters(sample_values, centroid_values, n_samples_per_cluster)
```

In [30]:
%%writefile generate_samples.py

import tensorflow as tf
import numpy as np

from functions import create_samples
from functions import plot_clusters

n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70

np.random.seed(seed)

centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)

model = tf.initialize_all_variables()
with tf.Session() as session:
    sample_values = session.run(samples)
    centroid_values = session.run(centroids)
    
plot_clusters(sample_values, centroid_values, n_samples_per_cluster)

Overwriting generate_samples.py


In [33]:
!python generate_samples.py

# Initialisation